In [24]:
from google.colab import files
files.download("/content/dashboard_state_month.csv")
files.download("/content/dashboard_district_month.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
district_month = (
    df.groupby(["month","state","district"], as_index=False)
    .agg(
        activity_total=("activity_total","sum"),
        enrol_total=("enrol_total","sum"),
        demo_total=("demo_total","sum"),
        bio_total=("bio_total","sum"),
        age_0_5=("age_0_5","sum"),
        age_5_17=("age_5_17","sum"),
        age_18_greater=("age_18_greater","sum"),
    )
)

district_month.to_csv("/content/dashboard_district_month.csv", index=False)

print("District rows:", district_month.shape)
district_month.head()


District rows: (7960, 10)


,month,state,district,activity_total,enrol_total,demo_total,bio_total,age_0_5,age_5_17,age_18_greater
0,2025-03-01,Andaman and Nicobar Islands,Andamans,209.0,0.0,0.0,209.0,0.0,0.0,0.0
1,2025-03-01,Andaman and Nicobar Islands,Nicobar,820.0,0.0,392.0,428.0,0.0,0.0,0.0
2,2025-03-01,Andaman and Nicobar Islands,North And Middle Andaman,1537.0,0.0,422.0,1115.0,0.0,0.0,0.0
3,2025-03-01,Andaman and Nicobar Islands,South Andaman,1922.0,0.0,524.0,1398.0,0.0,0.0,0.0
4,2025-03-01,Andhra Pradesh,Adilabad,10196.0,0.0,4340.0,5856.0,0.0,0.0,0.0


In [22]:
import numpy as np
import pandas as pd

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()

state_month = (
    df.groupby(["month","state"], as_index=False)
    .agg(
        activity_total=("activity_total","sum"),
        enrol_total=("enrol_total","sum"),
        demo_total=("demo_total","sum"),
        bio_total=("bio_total","sum"),
        age_0_5=("age_0_5","sum"),
        age_5_17=("age_5_17","sum"),
        age_18_greater=("age_18_greater","sum"),
    )
).sort_values(["state","month"])

state_month["prev_activity"] = state_month.groupby("state")["activity_total"].shift(1)
state_month["growth_pct"] = np.where(
    state_month["prev_activity"] > 0,
    (state_month["activity_total"] - state_month["prev_activity"]) / state_month["prev_activity"],
    np.nan
)

def safe_zscore(s):
    std = np.nanstd(s)
    if std == 0 or np.isnan(std):
        return np.nan
    return (s - np.nanmean(s)) / std

state_month["migration_index"] = state_month.groupby("month")["growth_pct"].transform(safe_zscore)

print("Final States/UTs in state_month:", state_month["state"].nunique())

state_month.to_csv("/content/dashboard_state_month.csv", index=False)
state_month.head()


Final States/UTs in state_month: 35


/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


,month,state,activity_total,enrol_total,demo_total,bio_total,age_0_5,age_5_17,age_18_greater,prev_activity,growth_pct,migration_index
0,2025-03-01,Andaman and Nicobar Islands,4488.0,0.0,1338.0,3150.0,0.0,0.0,0.0,NaN,NaN,NaN
35,2025-04-01,Andaman and Nicobar Islands,2744.0,0.0,0.0,2744.0,0.0,0.0,0.0,4488.0,-0.388592,0.068695
70,2025-05-01,Andaman and Nicobar Islands,1895.0,0.0,0.0,1895.0,0.0,0.0,0.0,2744.0,-0.309402,-0.777505
105,2025-06-01,Andaman and Nicobar Islands,1944.0,0.0,0.0,1944.0,0.0,0.0,0.0,1895.0,0.025858,0.194516
140,2025-07-01,Andaman and Nicobar Islands,3642.0,0.0,438.0,3204.0,0.0,0.0,0.0,1944.0,0.873457,0.253321


In [21]:
valid_states_uts = {
"Andhra Pradesh","Arunachal Pradesh","Assam","Bihar","Chhattisgarh","Goa","Gujarat",
"Haryana","Himachal Pradesh","Jharkhand","Karnataka","Kerala","Madhya Pradesh",
"Maharashtra","Manipur","Meghalaya","Mizoram","Nagaland","Odisha","Punjab","Rajasthan",
"Sikkim","Tamil Nadu","Telangana","Tripura","Uttar Pradesh","Uttarakhand","West Bengal",
"Delhi","Jammu and Kashmir","Ladakh",
"Andaman and Nicobar Islands","Chandigarh",
"Dadra and Nagar Haveli and Daman and Diu","Lakshadweep","Puducherry"
}

before = df["state"].nunique()
df = df[df["state"].isin(valid_states_uts)]
after = df["state"].nunique()

print("Before:", before, "After:", after)


Before: 52 After: 35


In [20]:
# Create a mapping from normalized name -> first seen clean name
mapping = {}
for original in states:
    n = original.lower().replace("&", "and")
    n = " ".join(n.split())
    mapping.setdefault(n, original)

# Apply normalized mapping
def canonical_state(x):
    x = str(x)
    n = x.lower().replace("&", "and")
    n = " ".join(n.split())
    return mapping.get(n, x)

df["state"] = df["state"].apply(canonical_state)

print("Unique states/UTs after canonical:", df["state"].nunique())


Unique states/UTs after canonical: 52


In [19]:
import pandas as pd

s = pd.Series(states)

# normalize for duplicate detection
norm = (
    s.str.lower()
     .str.replace("&", "and", regex=False)
     .str.replace(r"\s+", " ", regex=True)
     .str.strip()
)

dups = pd.DataFrame({"state": s, "norm": norm})
dups[dups.duplicated("norm", keep=False)].sort_values("norm")


,state,norm


In [18]:
states = sorted(df["state"].dropna().unique().tolist())
len(states), states


(52,
 ['Andaman and Nicobar Islands',
  'Andhra Pradesh',
  'Arunachal Pradesh',
  'Assam',
  'Balanagar',
  'Bihar',
  'Chandigarh',
  'Chhatisgarh',
  'Chhattisgarh',
  'Dadra And Nagar Haveli',
  'Dadra and Nagar Haveli and Daman and Diu',
  'Daman And Diu',
  'Darbhanga',
  'Delhi',
  'Goa',
  'Gujarat',
  'Haryana',
  'Himachal Pradesh',
  'Jaipur',
  'Jammu And Kashmir',
  'Jharkhand',
  'Karnataka',
  'Kerala',
  'Ladakh',
  'Lakshadweep',
  'Madanapalle',
  'Madhya Pradesh',
  'Maharashtra',
  'Manipur',
  'Meghalaya',
  'Mizoram',
  'Nagaland',
  'Nagpur',
  'Odisha',
  'Puducherry',
  'Punjab',
  'Puttenahalli',
  'Raja Annamalai Puram',
  'Rajasthan',
  'Sikkim',
  'Tamil Nadu',
  'Tamilnadu',
  'Telangana',
  'The Dadra And Nagar Haveli And Daman And Diu',
  'Tripura',
  'Uttar Pradesh',
  'Uttarakhand',
  'Uttaranchal',
  'West Bangal',
  'West Bengal',
  'West Bengli',
  'Westbengal'])

In [17]:
sorted(df["state"].unique())[:200]


['Andaman and Nicobar Islands',
 'Andhra Pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Balanagar',
 'Bihar',
 'Chandigarh',
 'Chhatisgarh',
 'Chhattisgarh',
 'Dadra And Nagar Haveli',
 'Dadra and Nagar Haveli and Daman and Diu',
 'Daman And Diu',
 'Darbhanga',
 'Delhi',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jaipur',
 'Jammu And Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Ladakh',
 'Lakshadweep',
 'Madanapalle',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Nagpur',
 'Odisha',
 'Puducherry',
 'Punjab',
 'Puttenahalli',
 'Raja Annamalai Puram',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Tamilnadu',
 'Telangana',
 'The Dadra And Nagar Haveli And Daman And Diu',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'Uttaranchal',
 'West Bangal',
 'West Bengal',
 'West Bengli',
 'Westbengal']

In [16]:
# Show suspicious state values
bad = df["state"].value_counts().reset_index()
bad.columns = ["state", "count"]

# show states that contain numbers OR are too long/odd
bad_states = bad[
    bad["state"].str.contains(r"\d", regex=True) |
    bad["state"].str.contains(r"[^a-zA-Z\s\-\(\)]", regex=True)
]

bad_states.head(50)


,state,count


In [13]:
import numpy as np

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()

state_month = (
    df.groupby(["month","state"], as_index=False)
    .agg(
        activity_total=("activity_total","sum"),
        enrol_total=("enrol_total","sum"),
        demo_total=("demo_total","sum"),
        bio_total=("bio_total","sum"),
        age_0_5=("age_0_5","sum"),
        age_5_17=("age_5_17","sum"),
        age_18_greater=("age_18_greater","sum"),
    )
).sort_values(["state","month"])

state_month["prev_activity"] = state_month.groupby("state")["activity_total"].shift(1)
state_month["growth_pct"] = np.where(
    state_month["prev_activity"] > 0,
    (state_month["activity_total"] - state_month["prev_activity"]) / state_month["prev_activity"],
    np.nan
)

def safe_zscore(s):
    std = np.nanstd(s)
    if std == 0 or np.isnan(std):
        return np.nan
    return (s - np.nanmean(s)) / std

state_month["migration_index"] = state_month.groupby("month")["growth_pct"].transform(safe_zscore)

print("Final states/UTs:", state_month["state"].nunique())
state_month.to_csv("/content/dashboard_state_month.csv", index=False)


Final states/UTs: 52


/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [12]:
# Remove rows where state is numeric or too short
df["state"] = df["state"].astype(str).str.strip()

df = df[~df["state"].str.match(r"^\d+$", na=False)]   # removes "100000" type
df = df[df["state"].str.len() > 2]                   # removes weird tiny junk

print("After cleaning, unique states/UTs:", df["state"].nunique())
df["state"].value_counts().head(20)


After cleaning, unique states/UTs: 52


,count
state,
Andhra Pradesh,290057
Tamil Nadu,286871
Uttar Pradesh,238561
West Bengal,230726
Maharashtra,228172
Karnataka,220936
Odisha,154178
Kerala,148838
Gujarat,140197


In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/uidai_merged_clean.csv", dtype={"state":str, "district":str})
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()

# Clean names
def clean_text(x):
    x = str(x).strip()
    x = x.replace("&", "and")
    x = " ".join(x.split())
    return x.title()

df["state"] = df["state"].apply(clean_text)
df["district"] = df["district"].apply(clean_text)

state_fix = {
    "Andaman And Nicobar Islands": "Andaman and Nicobar Islands",
    "Dadra And Nagar Haveli And Daman And Diu": "Dadra and Nagar Haveli and Daman and Diu",
    "Nct Of Delhi": "Delhi",
    "Delhi Nct": "Delhi",
    "Orissa": "Odisha",
    "Pondicherry": "Puducherry",
}
df["state"] = df["state"].replace(state_fix)

# Build state-month table
state_month = (
    df.groupby(["month","state"], as_index=False)
    .agg(
        activity_total=("activity_total","sum"),
        enrol_total=("enrol_total","sum"),
        demo_total=("demo_total","sum"),
        bio_total=("bio_total","sum"),
        age_0_5=("age_0_5","sum"),
        age_5_17=("age_5_17","sum"),
        age_18_greater=("age_18_greater","sum"),
    )
).sort_values(["state","month"])

state_month["prev_activity"] = state_month.groupby("state")["activity_total"].shift(1)
state_month["growth_pct"] = np.where(
    state_month["prev_activity"] > 0,
    (state_month["activity_total"] - state_month["prev_activity"]) / state_month["prev_activity"],
    np.nan
)

def safe_zscore(s):
    std = np.nanstd(s)
    if std == 0 or np.isnan(std):
        return np.nan
    return (s - np.nanmean(s)) / std

state_month["migration_index"] = state_month.groupby("month")["growth_pct"].transform(safe_zscore)

state_month.to_csv("/content/dashboard_state_month.csv", index=False)

print("Unique states/UTs:", state_month["state"].nunique())
state_month.head(10)


Unique states/UTs: 53


/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


,month,state,activity_total,enrol_total,demo_total,bio_total,age_0_5,age_5_17,age_18_greater,prev_activity,growth_pct,migration_index
192,2025-09-01,100000,12.0,12.0,0.0,0.0,0.0,0.0,12.0,NaN,NaN,NaN
235,2025-10-01,100000,1.0,1.0,0.0,0.0,0.0,1.0,0.0,12.0,-0.916667,-1.939466
277,2025-11-01,100000,11.0,11.0,0.0,0.0,0.0,0.0,11.0,1.0,10.000000,6.038822
320,2025-12-01,100000,196.0,194.0,2.0,0.0,0.0,0.0,194.0,11.0,16.818182,6.245503
0,2025-03-01,Andaman and Nicobar Islands,4488.0,0.0,1338.0,3150.0,0.0,0.0,0.0,NaN,NaN,NaN
38,2025-04-01,Andaman and Nicobar Islands,2744.0,0.0,0.0,2744.0,0.0,0.0,0.0,4488.0,-0.388592,-0.047589
76,2025-05-01,Andaman and Nicobar Islands,1895.0,0.0,0.0,1895.0,0.0,0.0,0.0,2744.0,-0.309402,-0.729676
114,2025-06-01,Andaman and Nicobar Islands,1944.0,0.0,0.0,1944.0,0.0,0.0,0.0,1895.0,0.025858,0.247911
153,2025-07-01,Andaman and Nicobar Islands,3642.0,0.0,438.0,3204.0,0.0,0.0,0.0,1944.0,0.873457,0.275887
193,2025-09-01,Andaman and Nicobar Islands,4595.0,227.0,1072.0,3296.0,209.0,18.0,0.0,3642.0,0.261669,-0.290519


In [10]:
import numpy as np

def safe_zscore(s):
    std = np.nanstd(s)
    if std == 0 or np.isnan(std):
        return np.nan
    return (s - np.nanmean(s)) / std

state_month["migration_index"] = state_month.groupby("month")["growth_pct"].transform(safe_zscore)


/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [9]:
import pandas as pd

df = pd.read_csv("/content/uidai_merged_clean.csv", dtype={"state":str, "district":str})
df["state"] = df["state"].astype(str).str.strip()
df["district"] = df["district"].astype(str).str.strip()

df[["state","district","pincode"]].head(10)


,state,district,pincode
0,Andaman & Nicobar Islands,Andamans,744101
1,Andaman and Nicobar Islands,Nicobar,744301
2,Andaman and Nicobar Islands,Nicobar,744301
3,Andaman and Nicobar Islands,Nicobar,744302
4,Andaman and Nicobar Islands,Nicobar,744303
5,Andaman and Nicobar Islands,Nicobar,744304
6,Andaman and Nicobar Islands,North And Middle Andaman,744201
7,Andaman and Nicobar Islands,North And Middle Andaman,744202
8,Andaman and Nicobar Islands,North And Middle Andaman,744202
9,Andaman and Nicobar Islands,North And Middle Andaman,744204


In [8]:
df[["state","district","pincode"]].head(10)

,state,district,pincode
0,Andaman and Nicobar Islands,Andamans,744101
1,Andaman and Nicobar Islands,Nicobar,744301
2,Andaman and Nicobar Islands,Nicobar,744301
3,Andaman and Nicobar Islands,Nicobar,744302
4,Andaman and Nicobar Islands,Nicobar,744303
5,Andaman and Nicobar Islands,Nicobar,744304
6,Andaman and Nicobar Islands,North And Middle Andaman,744201
7,Andaman and Nicobar Islands,North And Middle Andaman,744202
8,Andaman and Nicobar Islands,North And Middle Andaman,744202
9,Andaman and Nicobar Islands,North And Middle Andaman,744204


In [7]:
df.columns

Index(['date', 'state', 'district', 'pincode', 'age_0_5', 'age_5_17',
       'age_18_greater', 'demo_age_5_17', 'demo_age_17_', 'bio_age_5_17',
       'bio_age_17_', 'enrol_total', 'demo_total', 'bio_total',
       'activity_total', 'month'],
      dtype='object')

In [6]:
state_month = (
    df.groupby(["month", "state"], as_index=False)
    .agg(
        activity_total=("activity_total", "sum"),
        enrol_total=("enrol_total", "sum"),
        demo_total=("demo_total", "sum"),
        bio_total=("bio_total", "sum"),
        age_0_5=("age_0_5", "sum"),
        age_5_17=("age_5_17", "sum"),
        age_18_greater=("age_18_greater", "sum"),
    )
)

state_month = state_month.sort_values(["state", "month"])
state_month["prev_activity"] = state_month.groupby("state")["activity_total"].shift(1)
state_month["growth_pct"] = np.where(
    state_month["prev_activity"] > 0,
    (state_month["activity_total"] - state_month["prev_activity"]) / state_month["prev_activity"],
    np.nan
)

# Migration proxy index (z-score per month)
state_month["migration_index"] = state_month.groupby("month")["growth_pct"].transform(
    lambda s: (s - np.nanmean(s)) / np.nanstd(s) if np.nanstd(s) > 0 else np.nan
)

state_month.to_csv("/content/dashboard_state_month.csv", index=False)
state_month.head()


/usr/local/lib/python3.12/dist-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


,month,state,activity_total,enrol_total,demo_total,bio_total,age_0_5,age_5_17,age_18_greater,prev_activity,growth_pct,migration_index
192,2025-09-01,100000,12.0,12.0,0.0,0.0,0.0,0.0,12.0,NaN,NaN,NaN
235,2025-10-01,100000,1.0,1.0,0.0,0.0,0.0,1.0,0.0,12.0,-0.916667,-1.939466
277,2025-11-01,100000,11.0,11.0,0.0,0.0,0.0,0.0,11.0,1.0,10.000000,6.038822
320,2025-12-01,100000,196.0,194.0,2.0,0.0,0.0,0.0,194.0,11.0,16.818182,6.245503
0,2025-03-01,Andaman and Nicobar Islands,4488.0,0.0,1338.0,3150.0,0.0,0.0,0.0,NaN,NaN,NaN


In [5]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()


In [2]:
def clean_text(x):
    x = str(x).strip()
    x = x.replace("&", "and")
    x = " ".join(x.split())
    x = x.title()
    return x

df["state"] = df["state"].apply(clean_text)
df["district"] = df["district"].apply(clean_text)

# Fix common duplicates
state_fix = {
    "Andaman And Nicobar Islands": "Andaman and Nicobar Islands",
    "Dadra And Nagar Haveli And Daman And Diu": "Dadra and Nagar Haveli and Daman and Diu",
    "Nct Of Delhi": "Delhi",
    "Delhi Nct": "Delhi",
    "Orissa": "Odisha",
    "Pondicherry": "Puducherry",
}
df["state"] = df["state"].replace(state_fix)

print("Unique states/UTs:", df["state"].nunique())
df["state"].value_counts().head(20)


Unique states/UTs: 53


,count
state,
Andhra Pradesh,290057
Tamil Nadu,286871
Uttar Pradesh,238561
West Bengal,230726
Maharashtra,228172
Karnataka,220936
Odisha,154178
Kerala,148838
Gujarat,140197


In [1]:
import pandas as pd
import numpy as np
import zipfile

# ====== If you uploaded uidai_merged_clean.zip ======
zip_path = "/content/uidai_merged_clean.zip"

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall("/content/")

df = pd.read_csv("/content/uidai_merged_clean.csv")
print(df.shape)
df.head()


(2947614, 15)


,date,state,district,pincode,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,enrol_total,demo_total,bio_total,activity_total
0,2025-03-01,Andaman & Nicobar Islands,Andamans,744101,0.0,0.0,0.0,0.0,0.0,16.0,193.0,0.0,0.0,209.0,209.0
1,2025-03-01,Andaman and Nicobar Islands,Nicobar,744301,0.0,0.0,0.0,16.0,180.0,101.0,48.0,0.0,196.0,149.0,345.0
2,2025-03-01,Andaman and Nicobar Islands,Nicobar,744301,0.0,0.0,0.0,16.0,180.0,101.0,48.0,0.0,196.0,149.0,345.0
3,2025-03-01,Andaman and Nicobar Islands,Nicobar,744302,0.0,0.0,0.0,0.0,0.0,15.0,12.0,0.0,0.0,27.0,27.0
4,2025-03-01,Andaman and Nicobar Islands,Nicobar,744303,0.0,0.0,0.0,0.0,0.0,46.0,27.0,0.0,0.0,73.0,73.0


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['pincode'].plot(kind='hist', bins=20, title='pincode')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['demo_age_5_17'].plot(kind='hist', bins=20, title='demo_age_5_17')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['district'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='index', y='district', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_21['state'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_21, x='pincode', y='state', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_22['district'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_22, x='pincode', y='district', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)